In [9]:
import pandas as pd

from fastai.tabular.all import *

In [10]:
DATA_PATH = '../data/preprocessed_training_data.pkl'

In [11]:
df = pd.read_pickle(DATA_PATH)[:100000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  100000 non-null  object        
 1   timestamp           100000 non-null  datetime64[ns]
 2   campaignId          100000 non-null  object        
 3   platform            100000 non-null  object        
 4   softwareVersion     100000 non-null  object        
 5   sourceGameId        100000 non-null  object        
 6   country             100000 non-null  object        
 7   startCount          100000 non-null  int64         
 8   viewCount           100000 non-null  int64         
 9   clickCount          100000 non-null  int64         
 10  installCount        100000 non-null  int64         
 11  lastStart           93348 non-null   datetime64[ns]
 12  startCount1d        100000 non-null  int64         
 13  startCount7d        100000 non

In [33]:
class_count_df = df.groupby('install').count()['installCount']

n_0 = class_count_df[0]

# w_1 = (n_0 + n_1) / (2.0 * n_1)
# w_0 = (n_0 + n_1) / (2.0 * n_0)

class_count_df, n_1

(install
 0    98040
 1     1960
 Name: installCount, dtype: int64,
 Empty DataFrame
 Columns: [install]
 Index: [])

In [12]:
dls = TabularDataLoaders.from_df(df, y_names='install', 
                                 cat_names=['campaignId', 'platform', 'softwareVersion', 'sourceGameId', 
                                            'connectionType', 'deviceType', 'country'],
                                 cont_names=['startCount', 'viewCount', 'clickCount', 'installCount',
                                             'startCount1d', 'startCount7d', 'timeSinceLastStart'],
                                 procs=[Categorify, FillMissing, Normalize])

In [20]:
roc_auc = RocAuc()
balanced_accuracy = BalancedAccuracy()
learn = tabular_learner(dls, metrics=roc_auc)
# learn = tabular_learner(dls, metrics=BalancedAccuracy)

In [21]:
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,roc_auc_score,time
0,0.021454,0.019844,0.500000,00:46
1,0.021068,0.019621,0.500000,00:52
2,0.019203,0.020077,0.500000,00:47
